In [1]:
import snowflake.connector
import pandas as pd

# Set up Snowflake connection
conn = snowflake.connector.connect(
    user="ELII",
    password="Elii123456789!",
    account="CMZNSCB-MU47932",
    warehouse="COMPUTE_WH",
    database="WELLS",
    schema="MINERALS"
)
# query = "SELECT * FROM DI_TX_MINERAL_APPRAISALS_2023 LIMIT 10"
# query = "SELECT SUM(ARRAY_SIZE(SPLIT(API_10, ','))) AS total_well_ids FROM DI_TX_MINERAL_APPRAISALS_2023;"
# query = "SELECT t.*, TRIM(f.value) AS API_10 FROM MINERALS.DI_TX_MINERAL_APPRAISALS_2023 t,LATERAL FLATTEN(input => SPLIT(t.API_10, ',')) f;"

# Create a cursor
cur = conn.cursor()

# SQL Query to create the exploded table in Snowflake
create_table_query = """
CREATE TABLE MINERALS.DI_TX_MINERAL_APPRAISALS_2023_exploded AS
SELECT 
    t.*, 
    TRIM(f.value) AS API_10_exploded
FROM MINERALS.DI_TX_MINERAL_APPRAISALS_2023 t,
LATERAL FLATTEN(input => SPLIT(t.API_10, ',')) f;
"""

# Execute query
cur.execute(create_table_query)

# Close connection
cur.close()
conn.close()